# Predicting outputs and sending the data to MySQL

In [13]:
import pickle
import numpy as np
import pandas as pd
from sklearn import metrics
import pymysql

## Making predictions

In [2]:
model = pickle.load(open('model', 'rb'))

In [3]:
model

RandomForestClassifier(max_depth=8, max_features=15, min_samples_leaf=30,
                       min_samples_split=48, n_estimators=410, n_jobs=-1,
                       random_state=20)

In [10]:
test = pd.read_csv('Test_Data.csv')
test

,Reason1,Reason2,Reason3,Reason4,Month,Day,Transportation_Expense,Distance,Age,Workload,Hit_Target,Disciplinary_Failure,Education,Son,Social_Drinker,Social_Smoker,Pet,BMI
0,0,0,0,1,5,4,291,31,40,237656,99,0,0,1,1,0,1,25
1,1,0,0,0,6,6,246,25,41,377550,94,0,0,0,1,0,0,23
2,1,0,0,0,2,4,233,51,31,264249,97,0,1,1,1,0,8,21
3,0,0,0,1,1,4,179,51,38,313532,96,0,0,0,1,0,0,31
4,0,0,0,1,6,2,179,51,38,253957,95,0,0,0,1,0,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,0,0,0,1,1,3,225,26,28,308593,95,0,0,1,0,0,2,24
144,0,0,0,1,12,5,235,11,37,236629,93,0,1,1,0,0,1,29
145,0,0,0,1,4,2,179,51,38,239409,98,0,0,0,1,0,0,31
146,0,0,0,1,7,3,289,36,33,239554,97,0,0,2,1,0,1,30


In [11]:
pred = model.predict(test)
prob = model.predict_proba(test)[:, 1]

In [12]:
test['Prediction'] = pred
test['Probability'] = prob
test

,Reason1,Reason2,Reason3,Reason4,Month,Day,Transportation_Expense,Distance,Age,Workload,Hit_Target,Disciplinary_Failure,Education,Son,Social_Drinker,Social_Smoker,Pet,BMI,Prediction,Probability
0,0,0,0,1,5,4,291,31,40,237656,99,0,0,1,1,0,1,25,0,0.406268
1,1,0,0,0,6,6,246,25,41,377550,94,0,0,0,1,0,0,23,1,0.791293
2,1,0,0,0,2,4,233,51,31,264249,97,0,1,1,1,0,8,21,1,0.757568
3,0,0,0,1,1,4,179,51,38,313532,96,0,0,0,1,0,0,31,0,0.083849
4,0,0,0,1,6,2,179,51,38,253957,95,0,0,0,1,0,0,31,0,0.332363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,0,0,0,1,1,3,225,26,28,308593,95,0,0,1,0,0,2,24,0,0.106842
144,0,0,0,1,12,5,235,11,37,236629,93,0,1,1,0,0,1,29,0,0.457500
145,0,0,0,1,4,2,179,51,38,239409,98,0,0,0,1,0,0,31,0,0.298192
146,0,0,0,1,7,3,289,36,33,239554,97,0,0,2,1,0,1,30,1,0.669238


### Connecting MySQL

In [14]:
conn = pymysql.connect(database = 'absenteeism_outputs', user = 'root', password = 'St14@496')

In [15]:
cursor = conn.cursor()

In [18]:
query = 'select * from predicted_outputs;'

In [19]:
cursor.execute(query)

0

#### Creating the insert statement

In [71]:
insert = 'insert into predicted_outputs values '
for i in range(test.shape[0]):
    #print(insert)
    insert += '('
    #print(insert)
    for j in range(test.shape[1]):
        #print(str(test.iloc[i, j]) + ', ')
        insert += str(test.iloc[i, j]) + ', '
        #print(insert)
    #print(insert)
    insert = insert[:-2] + ');'
    print(insert)
    cursor.execute(insert)
    conn.commit()
    insert = 'insert into predicted_outputs values '

insert into predicted_outputs values (0, 0, 0, 1, 5, 4, 291, 31, 40, 237656, 99, 0, 0, 1, 1, 0, 1, 25, 0, 0.40626845360914043);
insert into predicted_outputs values (1, 0, 0, 0, 6, 6, 246, 25, 41, 377550, 94, 0, 0, 0, 1, 0, 0, 23, 1, 0.7912934954871534);
insert into predicted_outputs values (1, 0, 0, 0, 2, 4, 233, 51, 31, 264249, 97, 0, 1, 1, 1, 0, 8, 21, 1, 0.7575681108022727);
insert into predicted_outputs values (0, 0, 0, 1, 1, 4, 179, 51, 38, 313532, 96, 0, 0, 0, 1, 0, 0, 31, 0, 0.08384876803219007);
insert into predicted_outputs values (0, 0, 0, 1, 6, 2, 179, 51, 38, 253957, 95, 0, 0, 0, 1, 0, 0, 31, 0, 0.3323628204685655);
insert into predicted_outputs values (0, 0, 0, 1, 2, 3, 179, 26, 30, 302585, 99, 0, 1, 0, 0, 0, 0, 19, 0, 0.1564996014587587);
insert into predicted_outputs values (0, 0, 0, 1, 11, 4, 225, 26, 28, 306345, 93, 0, 0, 1, 0, 0, 2, 24, 0, 0.21712024846700018);
insert into predicted_outputs values (0, 0, 0, 1, 10, 3, 179, 51, 38, 253465, 93, 0, 0, 0, 1, 0, 0, 31, 0, 

insert into predicted_outputs values (0, 0, 0, 1, 12, 3, 225, 26, 28, 280549, 98, 0, 0, 1, 0, 0, 2, 24, 0, 0.20995038102486985);
insert into predicted_outputs values (0, 0, 0, 1, 1, 4, 118, 10, 37, 308593, 95, 0, 0, 0, 0, 0, 0, 28, 0, 0.11489086440016727);
insert into predicted_outputs values (0, 0, 0, 1, 9, 3, 155, 12, 34, 261756, 87, 0, 0, 2, 1, 0, 0, 25, 0, 0.4057124640979455);
insert into predicted_outputs values (1, 0, 0, 0, 5, 4, 289, 36, 33, 378884, 92, 0, 0, 2, 1, 0, 1, 30, 1, 0.8265762538765304);
insert into predicted_outputs values (0, 0, 0, 0, 10, 6, 118, 13, 50, 265017, 88, 1, 0, 1, 1, 0, 0, 31, 0, 0.38336498412403913);
insert into predicted_outputs values (1, 0, 0, 0, 7, 5, 189, 29, 33, 239554, 97, 0, 0, 2, 0, 0, 2, 25, 1, 0.5963814963798321);
insert into predicted_outputs values (0, 0, 0, 1, 3, 5, 225, 26, 28, 222196, 99, 0, 0, 1, 0, 0, 2, 24, 0, 0.18551749553337785);
insert into predicted_outputs values (1, 0, 0, 0, 3, 4, 260, 50, 36, 343253, 95, 0, 0, 4, 1, 0, 0, 23, 1,

In [72]:
conn.close()